In [15]:
import polars as pl
import re

from glob import glob

In [8]:
df_paths = r'F:\DataSpell\polars_library\Eurojackpot\*.csv'

In [5]:
file_paths = glob(df_paths)

In [9]:
df = pl.read_csv(df_paths)

In [17]:
def clean_date_string(date_string):
    return re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_string)

In [20]:
df = df.with_columns(
    pl.col('Result Date').map_elements(
        clean_date_string, pl.Utf8
    ).str.strptime(
        pl.Date, "%A %d %B %Y"
    )
)

In [31]:
df = df.with_columns(
    pl.col('Numbers').str.split_exact('  ', 7).struct.rename_fields(
        ['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Additional 1', 'Additional 2']
    ).alias('Numbers')
).unnest('Numbers')

In [34]:
df = df.drop('')

In [41]:
df.select(
    'First', 'Second', 'Third', 'Fourth', 'Fifth'
).unpivot()['value'].value_counts(sort=True)

value,count
str,u32
"""45""",22
"""30""",21
"""17""",18
"""11""",17
"""20""",17
…,…
"""33""",8
"""40""",8
"""50""",8


In [44]:
df.select(
    'Additional 1', 'Additional 2'
).unpivot()['value'].value_counts(sort=True)

value,count
str,u32
"""10""",32
"""1""",30
"""3""",23
"""12""",23
"""4""",22
…,…
"""7""",17
"""6""",17
"""9""",17


In [46]:
df.group_by(
    'First', 'Second', 'Third', 'Fourth', 'Fifth'
).len().sort(by='len', descending=True)

First,Second,Third,Fourth,Fifth,len
str,str,str,str,str,u32
"""2""","""3""","""4""","""21""","""45""",1
"""12""","""14""","""18""","""45""","""50""",1
"""15""","""17""","""27""","""33""","""45""",1
"""4""","""32""","""36""","""38""","""47""",1
"""2""","""4""","""23""","""30""","""40""",1
…,…,…,…,…,…
"""25""","""28""","""29""","""31""","""33""",1
"""3""","""4""","""13""","""20""","""21""",1
"""11""","""30""","""32""","""45""","""47""",1


In [47]:
df.group_by(
    'Additional 1', 'Additional 2'
).len().sort(by='len', descending=True)

Additional 1,Additional 2,len
str,str,u32
"""4""","""10""",6
"""1""","""3""",6
"""3""","""10""",6
"""1""","""11""",4
"""6""","""12""",4
…,…,…
"""4""","""8""",1
"""5""","""6""",1
"""1""","""7""",1
